# 라이브러리

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from PIL import Image
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset
from sklearn.model_selection import train_test_split
from pathlib import Path

# 데이터 경로 설정 및 기본 구조

In [ ]:
# 기본 경로 설정
RAW_DIR = Path('raw')
PROCESSED_DIR = Path('processed')
RESULTS_DIR = Path('results')

# 데이터 구조 정의
class StoryData:
    def __init__(self):
        self.story_id = None
        self.title = None
        self.text = None
        self.images = []
        self.scene_descriptions = []
        self.metadata = {}

# 이미지 전처리

In [ ]:
def preprocess_image(image_path):
    """이미지 전처리 함수
    - 이미지 크기 조정 (1024x1024) -> 모델 형식 (GPT-4o)
    - 정규화
    - 데이터 증강 
    """
    try:
        image = Image.open(image_path)
        # DALL-E 3 입력 형식에 맞게 조정
        image = image.resize((1024, 1024))
        # 이미지를 numpy 배열로 변환
        image_array = np.array(image)
        # 정규화 (0-1 범위로)
        image_array = image_array / 255.0
        return image_array
    except Exception as e: # 예외 처리
        print(f"이미지 처리 중 오류 발생: {e}")
        return None

def extract_image_features(image_array):
    """이미지 특징 추출 함수
    - 이미지의 주요 특징 추출
    - 스타일, 구도, 주요 객체 등 분석
    """
    # TODO: 이미지 특징 추출 로직 구현
    features = {
        'composition': None,  # 구도 분석
        'style': None,       # 스타일 분석
        'objects': [],       # 주요 객체 목록
        'colors': []         # 주요 색상 분포
    }
    return features

# 텍스트 전처리

In [ ]:
def preprocess_text(text):
    """텍스트 전처리 함수
    - 특수문자 처리
    - 문장 분리
    - 토큰화
    """
    # 기본 전처리
    text = text.strip()
    text = text.replace('\n', ' ')
    
    # 문장 단위로 분리
    sentences = text.split('.')
    sentences = [s.strip() for s in sentences if s.strip()]
    
    return sentences

def extract_scene_descriptions(text):
    """장면 설명 추출 함수
    - 텍스트에서 시각적 설명 부분 추출
    - 장면별 설명 분리
    """
    # TODO: 장면 설명 추출 로직 구현
    scenes = []
    return scenes

# 데이터셋 생성 및 통합

In [ ]:
def create_dataset(raw_data_path):
    """데이터셋 생성 함수"""
    dataset = []
    
    # 원본 데이터 읽기
    for story_dir in raw_data_path.glob('*'):
        if not story_dir.is_dir():
            continue
            
        story_data = StoryData()
        
        # 텍스트 데이터 읽기
        text_path = story_dir / 'story.txt'
        if text_path.exists():
            with open(text_path, 'r', encoding='utf-8') as f:
                story_data.text = f.read()
                
        # 이미지 처리
        for img_path in story_dir.glob('images/*.jpg'):
            processed_img = preprocess_image(img_path)
            if processed_img is not None:
                story_data.images.append(processed_img)
                
        dataset.append(story_data)
    
    return dataset

# 학습 데이터 준비

In [ ]:
def prepare_training_data(dataset):
    """학습 데이터 준비 함수
    - 데이터 분할 (학습/검증/테스트)
    - 데이터 포맷 변환
    """
    # 데이터 분할
    train_data, temp_data = train_test_split(dataset, test_size=0.3, random_state=42)
    val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)
    
    # 데이터 포맷 변환
    def format_data(data):
        return [{
            'text': item.text,
            'images': item.images,
            'scene_descriptions': item.scene_descriptions
        } for item in data]
    
    return (
        format_data(train_data),
        format_data(val_data),
        format_data(test_data)
    )

# 실행

In [ ]:
def main():
    # 데이터 전처리 파이프라인 실행
    raw_data_path = RAW_DIR
    processed_data = create_dataset(raw_data_path)
    
    # 학습/검증/테스트 데이터 준비
    train_data, val_data, test_data = prepare_training_data(processed_data)
    
    # 처리된 데이터 저장
    save_processed_data(processed_data, PROCESSED_DIR)
    
    # 데이터 검증
    validation_results = validate_processed_data(PROCESSED_DIR)
    
    # 결과 저장
    with open(RESULTS_DIR / 'preprocessing_results.json', 'w') as f:
        json.dump(validation_results, f, indent=2)

if __name__ == "__main__":
    main()